## Import libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from pyngrok import ngrok
import pandas as pd
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

## Set up ngrok to tunnel the local redirect URI

In [3]:
public_url = ngrok.connect(8888)
print(f"Public URL for OAuth callback: {public_url}")

Public URL for OAuth callback: NgrokTunnel: "https://f8a5-129-222-64-120.ngrok-free.app" -> "http://localhost:8888"


## Load environment variables from the .env file into jupyter notebook

In [5]:
load_dotenv()

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

print("Client ID and Client Secret are now loaded without showing them in code.")

Client ID and Client Secret are now loaded without showing them in code.


## Define the scope of access required for the Spotify API, specifically to modify public playlists
## Initialize the SpotifyOAuth object with the required credentials and scope for authorization

In [7]:
scope = 'playlist-modify-public'

sp_oauth = SpotifyOAuth(client_id=client_id,
                        client_secret=client_secret,
                        redirect_uri=redirect_uri,
                        scope=scope)

## Create a Spotipy client instance using the authentication manager (sp_oauth) for API access

In [9]:
sp = spotipy.Spotify(auth_manager=sp_oauth)

## Retrieve a list of similar tracks to a given track ID from the Spotify API, filtering the results based on specified popularity thresholds for both the artist and the track itself

In [11]:
def get_similar_tracks(track_id, limit=50, artist_popularity_limit=75, track_popularity_limit=60):
    """Fetches similar tracks to the given track_id, filtering by popularity."""
    recommendations = sp.recommendations(seed_tracks=[track_id], limit=limit)
    
    similar_tracks = []
    for track in recommendations['tracks']:
        artist_popularity = sp.artist(track['artists'][0]['id'])['popularity']
        track_popularity = track['popularity']
        
        if artist_popularity <= artist_popularity_limit and track_popularity <= track_popularity_limit:
            similar_tracks.append(track['id'])
    
    return similar_tracks

## Create Spotify playlist based on a DataFrame of favorite tracks by fetching similar songs and adding them to the newly created playlist, while providing a summary of the number of tracks added

In [13]:
def create_playlist_from_df(sp, username, df, playlist_name, playlist_description):
    """Creates a playlist from a dataframe of favorite tracks, fetching similar songs."""
    playlist = sp.user_playlist_create(user=username, name=playlist_name, public=True, description=playlist_description)
    
    all_similar_tracks = []
    for track_id in df['track_id']: 
        similar_tracks = get_similar_tracks(track_id)
        all_similar_tracks.extend(similar_tracks)
    
    track_uris = []
    for track_id in all_similar_tracks[:50]: 
        track_details = sp.track(track_id)
        track_uris.append(track_details['uri'])
    
    sp.playlist_add_items(playlist_id=playlist['id'], items=track_uris)  

    print(f"Playlist '{playlist_name}' created with {len(track_uris)} tracks.")
    
    return track_uris, playlist

## Load CSV files containing favorite tracks into pandas Dataframes

In [15]:
fav_house_tracks = pd.read_csv('fav_tracks_house_3.csv')
fav_reggaeton_tracks = pd.read_csv('fav_tracks_reggaeton_3.csv')

## Creates two playlists on Spotify—one for house tracks and one for reggaeton tracks 

In [17]:
house_tracks, house_playlist = create_playlist_from_df(sp, username='1272030812', df=fav_house_tracks, 
                                         playlist_name="Similar House Tracks", 
                                         playlist_description="50 songs similar to my favorite house tracks")

reggaeton_tracks, reggaeton_playlist = create_playlist_from_df(sp, username='1272030812', df=fav_reggaeton_tracks, 
                                             playlist_name="Similar Reggaeton Tracks", 
                                             playlist_description="50 songs similar to my favorite reggaeton tracks")

# Print playlist URLs
print(f"\nHouse Playlist URL: {house_playlist['external_urls']['spotify']}")
print(f"Reggaeton Playlist URL: {reggaeton_playlist['external_urls']['spotify']}")

Playlist 'Similar House Tracks' created with 43 tracks.
Playlist 'Similar Reggaeton Tracks' created with 8 tracks.

House Playlist URL: https://open.spotify.com/playlist/5ukSsNuRz87ZB6mobhdFlL
Reggaeton Playlist URL: https://open.spotify.com/playlist/1Z3c51s65ZNTeHeZCkoyLf


## Display the tracks that were added to the house and reggaeton playlists

In [19]:
def print_added_tracks(track_uris):
    """Prints the names of tracks given their URIs."""
    for uri in track_uris:
        track_details = sp.track(uri)
        print(f"Added Track: {track_details['name']} by {', '.join(artist['name'] for artist in track_details['artists'])}")

print("\nTracks added to the House Playlist:")
print_added_tracks(house_tracks)

print("\nTracks added to the Reggaeton Playlist:")
print_added_tracks(reggaeton_tracks)


Tracks added to the House Playlist:
Added Track: What To Do - &ME Remix by Guy Gerber, &ME
Added Track: Alegria by ARYMÉ, Osfur
Added Track: Pantheon by Argy, Goom Gum
Added Track: Dare Your Move by MAXI MERAKI
Added Track: Crazy by Datta Boy, Malachiii
Added Track: Mwaki by Zerb, Sofiya Nzau
Added Track: Povoada - Remix by Sued Nunes, Maz, Antdot
Added Track: Mood by Makar
Added Track: All You Children by Jamie xx, The Avalanches
Added Track: Heartless by WhoMadeWho, Kölsch
Added Track: Next To Me by Malachiii
Added Track: Electric Feel by James Carter, Malou
Added Track: My Love for You (Yebba's Heartbreak) by Marten Lou
Added Track: Second Chances by Wakyin, Chasity, PitchBLK
Added Track: Bloody Samaritan by Double MZK
Added Track: Pantheon by Argy, Goom Gum
Added Track: Povoada - Remix by Sued Nunes, Maz, Antdot
Added Track: Wish We Could by Nico de Andrea, Darla Jade, Syon
Added Track: Alegria by ARYMÉ, Osfur
Added Track: Voyage Voyage by Francis Mercier, Mont Rouge, Coco
Added T